# Preperation of the dataset exported from limesurvey

## Helper functions and imports

In [11]:
# Import necessary libraries
import pandas as pd

# Function to convert A1L-A7H to TLX numbers
# The function takes a string as input and returns a number as output


def convert_tlx_likert(x):
    if len(x) != 3 or x[0] != 'A' or not x[1].isdigit() or x[2] not in ('L', 'M', 'H'):
        raise ValueError("Invalid input format")

    number = int(x[1])
    suffix = x[2]

    suffix_values = {'L': 0, 'M': 5, 'H': 10}
    value = (number - 1) * 15 + suffix_values[suffix]

    return value

# Function to convert A1-A7 to numbers
# A1 to 1, A2 to 2, A3 to 3, etc.
# The function takes a string as input and returns a number as output


def convert_ueq_likert(x):
    if pd.isnull(x):
        return x
    else:
        return int(x[1])

# Function to convert A1 to yes and A2 to no
# The function takes a string as input and returns a boolean as output


def convert_yes_no(x):
    if pd.isnull(x):
        return x
    else:
        if x[1] == '1':
            return True
        elif x[1] == '2':
            return False
        else:
            raise ValueError(
                "Invalid input format. The input string should end with '1' or '2'.")

# Function to convert A1 to male, A2 to female and A3 to diverse


def convert_sex(x):
    if pd.isnull(x):
        return x
    else:
        if x[1] == '1':
            return 'male'
        elif x[1] == '2':
            return 'female'
        elif x[1] == '3':
            return 'diverse'
        else:
            raise ValueError("Invalid input format.")

# Function to convert A1 to male, A2 to female and A3 to diverse


def convert_dominant_hand(x):
    if pd.isnull(x):
        return x
    else:
        if x[1] == '1':
            return 'right'
        elif x[1] == '2':
            return 'left'
        elif x[1] == '3':
            return 'both'
        elif x[1] == '4':
            return 'neither'
        else:
            raise ValueError("Invalid input format.")

In [12]:

# Read data
test_results_df = pd.read_csv('./raw_data/test_results.csv')
survey_results_df = pd.read_csv('./raw_data/survey_results.csv')
calibration_df = pd.read_csv('./raw_data/calibration_results.csv')

# Fix column typo from lime survey
# iterate through the pre_results and change TXL to TLX
for col in survey_results_df.columns:
    if col.startswith('TXL'):
        new_col = col.replace('TXL', 'TLX')
        survey_results_df.rename(columns={col: new_col}, inplace=True)

# Initialize the final data list
final_data = []

# Iterate through the mapping
for _, calibration_row in calibration_df.iterrows():
    # Get data for corresponding SubjectID and Condition
    subject_id = calibration_row['Subject ID']
    condition = calibration_row['Group']
    data_row = test_results_df[(test_results_df['SubjectID'] == subject_id)
                       & (test_results_df['Condition'] == condition)].iloc[0]

   # Get the UEQ and TLX mapping from the mapping row
    mapping_number = calibration_row['Mapping']

    # Build the column names
    ueq_cols = [f"UEQS{mapping_number}[SQ00{i}]" for i in range(1, 9)]
    tlx_cols = [f"TLX{i}{mapping_number}[SQ001]" for i in range(1, 7)]

    other_cols = ["DS01", "DS02", "DS03", "IDRND",
                  "D01", "D02", "D03", "D04", "D05", "D06","EMSEmpfindung1", "EMSEmpfindung2", "EMSEmpfindung3", "EMSEmpfindung4"]
    calibration_cols = ["Calibration Arm", "Calibration Hand"]

    # Get the corresponding row from pre_results
    pre_results_row = survey_results_df[survey_results_df['IDRND']
                                     == subject_id].iloc[0]

    # Extract the UEQ and TLX values
    ueq_values = [pre_results_row[col] for col in ueq_cols]
    tlx_values = [pre_results_row[col] for col in tlx_cols]
    other_values = [pre_results_row[col]
                    for col in other_cols if pre_results_row[col] is not None]
    calibration_values = [calibration_row[col]
                    for col in calibration_cols if calibration_row[col] is not None]

    # Combine into a single row
    final_row = data_row.tolist() + ueq_values + tlx_values + other_values + calibration_values
    final_data.append(final_row)

# Create the final DataFrame
final_columns = list(test_results_df.columns) + [f'UEQ Item {i + 1}' for i in range(8)] + [
    f'TLX Item {i + 1}' for i in range(6)] + other_cols + calibration_cols
final_df = pd.DataFrame(final_data, columns=final_columns)

In [13]:
# Define column names
column_names = {
    "DS01": "DataProtectionAgreement",
    "DS02": "LiabilityWaiverSigned",
    "DS03": "HasPacemaker",
    "IDRND": "ID",
    "D01": "Age",
    "D02": "Gender",
    "D03": "ExperienceWithEMS",
    "D04": "ExperienceWithVR",
    "D05": "HandDominance",
    "D06": "Colorblindness",
    "UEQ Item 1": "UEQ_behindernd_unterstützend",
    "UEQ Item 2": "UEQ_kompliziert_einfach",
    "UEQ Item 3": "UEQ_ineffizient_effizient",
    "UEQ Item 4": "UEQ_verwirrend_übersichtlich",
    "UEQ Item 5": "UEQ_langweilig_spannend",
    "UEQ Item 6": "UEQ_uninteressant_interessant",
    "UEQ Item 7": "UEQ_konventionell_originell",
    "UEQ Item 8": "UEQ_herkömmlich_neuartig",
    # TLX
    "TLX Item 1": "TLX_MentalDemand",
    "TLX Item 2": "TLX_PhysicalDemand",
    "TLX Item 3": "TLX_TimeDemand",
    "TLX Item 4": "TLX_Performance",
    "TLX Item 5": "TLX_Effort",
    "TLX Item 6": "TLX_Frustration",

    "EMSEmpfindung1": "EMS_Empfindung1",
    "EMSEmpfindung2": "EMS_Empfindung2",
    "EMSEmpfindung3": "EMS_Empfindung3",
    "EMSEmpfindung4": "EMS_Empfindung4",

    "interviewtime": "InterviewTime"
}

# Apply the conversion functions to the relevant columns
for col in final_df.columns:
    if 'UEQ' in col:
        final_df[col] = final_df[col].apply(convert_ueq_likert)
    elif 'TLX' in col or 'TXL' in col:
        final_df[col] = final_df[col].apply(convert_tlx_likert)
    elif col in ['DS01', 'DS02', 'DS03', 'D03', 'D04', 'D06']:
        final_df[col] = final_df[col].apply(convert_yes_no)
    elif col == 'D02':
        final_df[col] = final_df[col].apply(convert_sex)
    elif col == 'D05':
        final_df[col] = final_df[col].apply(convert_dominant_hand)




# Rename columns
final_df.rename(columns=column_names, inplace=True)

final_df.to_csv('final_data.csv', index=False)

In [14]:
final_df

,SubjectID,nickname,Condition,ErrorCount,CorrectCount,TotalTime,AvgReactionTime,TimeInZoneHigh,TimeInZoneLow,UEQ_behindernd_unterstützend,...,ExperienceWithEMS,ExperienceWithVR,HandDominance,Colorblindness,EMS_Empfindung1,EMS_Empfindung2,EMS_Empfindung3,EMS_Empfindung4,Calibration Arm,Calibration Hand
0,1,Kathy,Control,1,49,105.41660,2.108334,18.178380,8.187508,4,...,True,True,right,False,kribbeln und zucken,"EMS Feedback ist direkter, da das Feedback dir...","Ist anfangs ungewohnt, vor allem bei Force Fee...",Es hat zur Verbesserung der Fehlervermeidung g...,11,50
1,1,Kathy,Visual,1,49,110.49770,2.209951,8.285627,17.262210,6,...,True,True,right,False,kribbeln und zucken,"EMS Feedback ist direkter, da das Feedback dir...","Ist anfangs ungewohnt, vor allem bei Force Fee...",Es hat zur Verbesserung der Fehlervermeidung g...,11,50
2,1,Kathy,EMS,0,50,109.28630,2.185727,7.504075,9.106119,6,...,True,True,right,False,kribbeln und zucken,"EMS Feedback ist direkter, da das Feedback dir...","Ist anfangs ungewohnt, vor allem bei Force Fee...",Es hat zur Verbesserung der Fehlervermeidung g...,11,50
3,1,Kathy,EMSVisual,0,50,113.26840,2.265368,19.976550,11.667920,7,...,True,True,right,False,kribbeln und zucken,"EMS Feedback ist direkter, da das Feedback dir...","Ist anfangs ungewohnt, vor allem bei Force Fee...",Es hat zur Verbesserung der Fehlervermeidung g...,11,50
4,2,Julien,Visual,1,49,109.91370,2.198277,16.727360,21.942710,5,...,True,True,right,False,Da ich mich eher auf das visuelle konzentriere...,Es gibt eine direkte körperliche Reaktion wied...,"Es fühlt sich an wie ein kribbeln, je nach Stä...",Für mich hat es beim ersten Verwenden eher zu ...,4,50
5,2,Julien,EMSVisual,0,50,134.84670,2.696932,11.665770,4.734380,4,...,True,True,right,False,Da ich mich eher auf das visuelle konzentriere...,Es gibt eine direkte körperliche Reaktion wied...,"Es fühlt sich an wie ein kribbeln, je nach Stä...",Für mich hat es beim ersten Verwenden eher zu ...,4,50
6,2,Julien,Control,0,50,95.68494,1.913697,23.236170,6.948138,4,...,True,True,right,False,Da ich mich eher auf das visuelle konzentriere...,Es gibt eine direkte körperliche Reaktion wied...,"Es fühlt sich an wie ein kribbeln, je nach Stä...",Für mich hat es beim ersten Verwenden eher zu ...,4,50
7,2,Julien,EMS,0,50,107.76950,2.155392,35.572220,4.096400,4,...,True,True,right,False,Da ich mich eher auf das visuelle konzentriere...,Es gibt eine direkte körperliche Reaktion wied...,"Es fühlt sich an wie ein kribbeln, je nach Stä...",Für mich hat es beim ersten Verwenden eher zu ...,4,50
8,3,Fabi,EMSVisual,4,46,174.49660,3.489918,31.803900,13.151140,2,...,False,False,right,False,Anstregend,NaN,"neu, gestresster","deutlich, schneller",6,28
9,3,Fabi,EMS,3,47,114.13270,2.282655,22.603920,10.604920,3,...,False,False,right,False,Anstregend,NaN,"neu, gestresster","deutlich, schneller",6,28
